 # NLP Lab 5

---

## Mateusz Praski

In [59]:
from transformers import pipeline, set_seed, AutoModel, AutoTokenizer
from morfeusz2 import Morfeusz

import openai
import re
import os

In [60]:
set_seed(42)

In [61]:
morf = Morfeusz()

# Download three Polish models from the Huggingface repository. These should be regular language models, which were not fine-tuned. E.g. HerBERT and papuGaPT2 are good examples.

I selected two Polish mask-filling models - BART and HerBERT, and also one multilingual model xml-roBERTa-large to check how multilingual models perform in comparison to trained one for specific language


In [62]:
papuga = pipeline('text-generation', model='flax-community/papuGaPT2-large')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [63]:
papuga("Idę z <mask> na spacer")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Idę z <mask> na spacer? Wypchaj się do tyłu z takim.\nTak, tak, tak! Jeśli nie lubisz być agresywny, to ta pozycja przypadnie ci do gustu - może to oznaczać: bycie agresywnym, na przykład,'}]

In [64]:
herbert_base = pipeline(task="fill-mask", model='allegro/herbert-base-cased')
herbert_answer = lambda x: sorted([(x['token_str'], x['score']) for x in herbert_base(x)], key=lambda x: x[1], reverse=True)

In [65]:
sorted([(x['token_str'], x['score']) for x in herbert_base("Warszawa to największe <mask>")], key=lambda x: x[1], reverse=True)

[('miasto', 0.8534282445907593),
 ('…', 0.02993311546742916),
 ('miasta', 0.024025341495871544),
 ('lotnisko', 0.02113961987197399),
 ('centrum', 0.015282508917152882)]

In [66]:
herbert_answer("Warszawa to największe <mask>")

[('miasto', 0.8534282445907593),
 ('…', 0.02993311546742916),
 ('miasta', 0.024025341495871544),
 ('lotnisko', 0.02113961987197399),
 ('centrum', 0.015282508917152882)]

In [67]:
roberta = pipeline("fill-mask", "xlm-roberta-large")
roberta_answer = lambda x: sorted([(x['token_str'], x['score']) for x in roberta(x)], key=lambda x: x[1], reverse=True)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [68]:
roberta_answer("Warszawa to największe <mask>")

[('miasto', 0.9453091025352478),
 ('miasta', 0.030545899644494057),
 ('centrum', 0.012399893254041672),
 ('Miasto', 0.0023689093068242073),
 ('...', 0.0016839336603879929)]

In [69]:
bart = pipeline("fill-mask", "sdadas/polish-bart-base")
bart_answer = lambda x: sorted([(x['token_str'], x['score']) for x in bart(x)], key=lambda x: x[1], reverse=True)

In [70]:
bart_answer("Warszawa to największe <mask>")

[('miasto', 0.6284970641136169),
 ('miasta', 0.0701904147863388),
 ('centrum', 0.046382322907447815),
 ('polskie', 0.04031280800700188),
 ('...', 0.03190264850854874)]

In [71]:
morf.analyse('pies psa')

[(0,
  1,
  ('pies', 'pies:Sm1', 'subst:sg:nom:m1', ['nazwa_pospolita'], ['pot.'])),
 (0, 1, ('pies', 'pies:Sm2', 'subst:sg:nom:m2', ['nazwa_pospolita'], [])),
 (1,
  2,
  ('psa', 'pies:Sm1', 'subst:sg:gen.acc:m1', ['nazwa_pospolita'], ['pot.'])),
 (1, 2, ('psa', 'pies:Sm2', 'subst:sg:gen.acc:m2', ['nazwa_pospolita'], []))]

In [72]:
def get_predicted_declension(word):
    return set([z
                    for x in morf.analyse(word)
                    if x[2][2].split(':')[0] == 'subst'
                    for z in x[2][2].split(':')[2].split('.')])

In [73]:
def get_nom_declension(word):
    rs = [x[2][1].split(':')[0] for x in morf.analyse(word)]
    if len(rs) != 1:
        # print(f"!!!! {rs}")
        pass
    return rs[0]

In [74]:
get_predicted_declension('klubem')

{'inst'}

In [75]:
get_nom_declension('klubem')

'klub'

In [76]:
polish_test = {
    # declension -> (query, expected, nom word)
    'mianownik': ('Warszawa to największe <mask>', 'nom', 'miasto'),
    'dopełniacz': ('Mój kolega nie ma <mask>. Musi przez to jeździć pociągiem do pracy', 'gen', 'auto'),
    'celownik': ('W <mask> na lepsze życie, kupiłem los na loterii', 'dat', 'nadzieja'),
    'biernik': ('Jestem roztrzepany, więc zapomniałem kupić <mask>', 'acc', 'ser'),
    'narzędnik': ('Idę z <mask> na spacer', 'inst', 'pies'),
    'miejscownik': ('Gdy jest mi źle, myślę o <mask>', 'loc', 'kot'),
    'wołacz': ('<mask>, Ojczyzno Moja!', 'voc', 'litwa')
}

In [77]:
def test(model):
    answers = {}
    
    for dec, (query, expected, base) in polish_test.items():
        ans = model(query)[0][0]
        # print(f">>{ans}<<")
        actual_dec = get_predicted_declension(ans.lower())
        actual_word = get_nom_declension(ans)
        
        points = 0
        if expected in actual_dec:
            points += 10
        if actual_word == base:
            points += 2
        
        answers[dec] = {
            'answer': ans,
            'dec': actual_dec,
            'word': actual_word,
            'points': points,
            'text': re.sub("<mask>", ans, query)
        }
    
    return answers

In [114]:
for model_name, model in {'herbert': herbert_answer, 'roberta': roberta_answer, 'bart': bart_answer}.items():
    res = test(model)
    print("============================")
    print(f"Model   {model_name}")
    print(f"Points  {sum([x['points'] for x in res.values()])}")
    for dec, rs in res.items():
        print(f"{dec} => {rs['text']}")
        

Model   herbert
Points  42
mianownik => Warszawa to największe miasto
dopełniacz => Mój kolega nie ma samochodu. Musi przez to jeździć pociągiem do pracy
celownik => W oczekiwaniu na lepsze życie, kupiłem los na loterii
biernik => Jestem roztrzepany, więc zapomniałem kupić .
narzędnik => Idę z Tobą na spacer
miejscownik => Gdy jest mi źle, myślę o sobie
wołacz => Witaj, Ojczyzno Moja!
Model   roberta
Points  42
mianownik => Warszawa to największe miasto
dopełniacz => Mój kolega nie ma samochodu. Musi przez to jeździć pociągiem do pracy
celownik => W drodze na lepsze życie, kupiłem los na loterii
biernik => Jestem roztrzepany, więc zapomniałem kupić .
narzędnik => Idę z ним na spacer
miejscownik => Gdy jest mi źle, myślę o Bogu
wołacz => Polska, Ojczyzno Moja!
Model   bart
Points  42
mianownik => Warszawa to największe miasto
dopełniacz => Mój kolega nie ma pracy. Musi przez to jeździć pociągiem do pracy
celownik => W poszukiwaniu na lepsze życie, kupiłem los na loterii
biernik => Jeste

# 4. Devise a method to test long-range relationships such as gender. E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [79]:
fill_sentence = lambda model, sentence: re.sub("<mask>", model(sentence)[0][0], sentence)

In [80]:
sen_1 = "Wczoraj miałem pojechać do rodziców, ale w natłoku rzeczy <mask> o tym"

In [81]:
fill_sentence(herbert_answer, sen_1)

'Wczoraj miałem pojechać do rodziców, ale w natłoku rzeczy myślałem o tym'

In [82]:
fill_sentence(roberta_answer, sen_1)

'Wczoraj miałem pojechać do rodziców, ale w natłoku rzeczy nie o tym'

In [83]:
fill_sentence(bart_answer, sen_1)

'Wczoraj miałem pojechać do rodziców, ale w natłoku rzeczy nie o tym'

In [84]:
sen_2 = 'Ala uwielbiała swojego kota, do tego stopnia, że w podczas podróży <mask> do niego'

In [85]:
fill_sentence(herbert_answer, sen_2)

'Ala uwielbiała swojego kota, do tego stopnia, że w podczas podróży chodziła do niego'

In [86]:
fill_sentence(roberta_answer, sen_2)

'Ala uwielbiała swojego kota, do tego stopnia, że w podczas podróży wraca do niego'

In [87]:
fill_sentence(bart_answer, sen_2)

'Ala uwielbiała swojego kota, do tego stopnia, że w podczas podróży zawsze do niego'

In [123]:
sen_3 = "Aleksandra była pierworoczną cheerleaderką drużyny licealnej. Kamil kapitan drużyny piłki nożnej, był w niej zauroczony od pierwszego wejrzenia. Niestety dla szans Kamila, nic do niego nie <mask>"

In [124]:
fill_sentence(herbert_answer, sen_3)

'Aleksandra była pierworoczną cheerleaderką drużyny licealnej. Kamil kapitan drużyny piłki nożnej, był w niej zauroczony od pierwszego wejrzenia. Niestety dla szans Kamila, nic do niego nie …'

In [125]:
fill_sentence(roberta_answer, sen_3)

'Aleksandra była pierworoczną cheerleaderką drużyny licealnej. Kamil kapitan drużyny piłki nożnej, był w niej zauroczony od pierwszego wejrzenia. Niestety dla szans Kamila, nic do niego nie ...'

In [126]:
fill_sentence(bart_answer, sen_3)

'Aleksandra była pierworoczną cheerleaderką drużyny licealnej. Kamil kapitan drużyny piłki nożnej, był w niej zauroczony od pierwszego wejrzenia. Niestety dla szans Kamila, nic do niego nie dociera'

# 5. Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [92]:
q_1 = "<mask> wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza"

In [93]:
fill_sentence(herbert_answer, q_1)

'Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza'

In [94]:
fill_sentence(roberta_answer, q_1)

'Olej wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza'

In [95]:
fill_sentence(bart_answer, q_1)

'W wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza'

In [96]:
q_2 = "<mask> 13 grudnia 1981 roku wprowadził w Polsce stan wojenny"

In [97]:
fill_sentence(herbert_answer, q_2)

'Sejm 13 grudnia 1981 roku wprowadził w Polsce stan wojenny'

In [98]:
fill_sentence(roberta_answer, q_2)

'- 13 grudnia 1981 roku wprowadził w Polsce stan wojenny'

In [99]:
fill_sentence(bart_answer, q_2)

'13 13 grudnia 1981 roku wprowadził w Polsce stan wojenny'

In [100]:
q_3 = "Iloczyn dwóch liczb ujemnych jest liczbą <mask>"

In [101]:
fill_sentence(herbert_answer, q_3)

'Iloczyn dwóch liczb ujemnych jest liczbą :'

In [102]:
fill_sentence(roberta_answer, q_3)

'Iloczyn dwóch liczb ujemnych jest liczbą :'

In [103]:
fill_sentence(bart_answer, q_3)

'Iloczyn dwóch liczb ujemnych jest liczbą dodat'

# 6. Check zero-shot learning capabilites of the models. Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie [MASK]" Try different prompts, to see if they make any difference.

In [104]:
predict_sentiment = lambda model, prompt, query: model(re.sub("<X>", query, prompt))

In [105]:
prompt_1 = "`<X>`. Wypowiedź ta miała zdecydowanie charakter <mask>"
prompt_2 = "Recenzje filmowe mogą być pozytywne, negatywne, bądź neutralne. Dla przykładu ta wypowiedź `<X>` jest <mask>"
prompt_3 = "Moim zdaniem wypowiedź `<X>` wypowiada się <mask> na temat filmu"

In [111]:
review_0 = "Ten film to był kiler. Nie mogłem się oderwać od ekranu."
review_1 = "Wojna żeńsko-męska to film, który wgl nie powinien powstać. Pani Samson (o dziwo - psycholożka, pisarka, dziennikarka, zupełnie jak główna bohaterka filmu, Barbara Patrycka) stworzyła beznadziejny scenariusz, a pan Palkowski przy dobrym Rezerwacie stracił w oczach widzów. "
review_2 = "Hipnotyzujący i olśniewający wizualnie. Reymont odczytany na nowo."
review_3 = "Główny bohater jest wręcz obleśnie lubialny. Niepotrzebna próba wrzucenia lekcji o walce z systemem, ale wspaniale ogląda się zachowania postaci, charaktery."
review_4 = "To było tak złe, że mam ochotę podnieść oceny wszystkim filmom, które oceniłam na 1. Arcydzieło bezguścia. Ostrzegam: nie oglądajcie tego na trzeźwo!"

In [107]:
re.sub("<X>", review_1, prompt_3)

'Moim zdaniem wypowiedź `Wojna żeńsko-męska to film, który wgl nie powinien powstać. Pani Samson (o dziwo - psycholożka, pisarka, dziennikarka, zupełnie jak główna bohaterka filmu, Barbara Patrycka) stworzyła beznadziejny scenariusz, a pan Palkowski przy dobrym Rezerwacie stracił w oczach widzów. ` wypowiada się <mask> na temat filmu'

In [113]:
for idr, (review, rs) in enumerate(zip([review_0, review_1, review_2, review_3, review_4], ['pozytywna', 'negatwyna', 'pozytywna', 'pozytywna', 'negatywna'])):
    print(f"======= {rs} ==========")
    for model_name, model in {'bart': bart_answer, 'herbert': herbert_answer, 'roberta': roberta_answer}.items():
        for idp, prompt in enumerate([prompt_1, prompt_2, prompt_3]):
            res = predict_sentiment(model, prompt, review)[0][0]
            print(f"{model_name:>10} - prompt {idp + 1} - review {idr} -> {res}")

======= pozytywna ==========
      bart - prompt 1 - review 0 -> ironi
      bart - prompt 2 - review 0 -> bardzo
      bart - prompt 3 - review 0 -> w
   herbert - prompt 1 - review 0 -> .
   herbert - prompt 2 - review 0 -> …
   herbert - prompt 3 - review 0 -> negatywnie
   roberta - prompt 1 - review 0 -> ...
   roberta - prompt 2 - review 0 -> ...
   roberta - prompt 3 - review 0 -> idealnie
======= negatwyna ==========
      bart - prompt 1 - review 1 -> ironi
      bart - prompt 2 - review 1 -> to
      bart - prompt 3 - review 1 -> w
   herbert - prompt 1 - review 1 -> .
   herbert - prompt 2 - review 1 -> .
   herbert - prompt 3 - review 1 -> krytycznie
   roberta - prompt 1 - review 1 -> .
   roberta - prompt 2 - review 1 -> ...
   roberta - prompt 3 - review 1 -> źle
======= pozytywna ==========
      bart - prompt 1 - review 2 -> proro
      bart - prompt 2 - review 2 -> bardzo
      bart - prompt 3 - review 2 -> w
   herbert - prompt 1 - review 2 -> .
   herbert - prompt 2

# Answer the following questions (2 points):

## Which of the models produced the best results?

I think that the best model by far is herBERT. Firstly, it hadn't had any strange response (in comparison to the "partially spelled" words by Bart, and Cyrillic from roBERTa. Also, his responses had the most sense.

## Was any of the models able to capture Polish grammar?

Both roBERTa-large and herBERT filled the answers with proper grammar and flexion in almost all the cases.

BART very often generated weird responses (e.g. `humo`, `'W wrze w...` or `jest liczbą dodat`)

## Was any of the models able to capture long-distant relationships between the words?

With the 4th task, herBERT was the best, answering all the questions correctly (although he chose some words that didn't need proper verb gender)

BART and roBERTa performed very similar.

## Was any of the models able to capture world knowledge?

Again the closest one was herBERT, where he answered correctly water, and tried his best with martial law.

On the other hand, BART as the only one, answered what's the product of tho negative values (although he said `dodat` not `dodatnia`)

## Was any of the models good at doing zero-shot classification?

The best one in my opinion was herBERT with 3rd prompt `Moim zdaniem wypowiedź `<X>` wypowiada się <mask> na temat filmu`

On the other hand roBERTa was classifying the reviews the same way as student which is not prepared for the test (like `this text reviews the movie perfectly`)

Most of the time, models just put out single dot as the answer (I'm not sure what does it mean)

## What are the most striking errors made by the models?

- roBERTa answering one question using Russian instead of Polish
- Plenty of random words generated by BART
